In [1]:
import pymupdf
import re
import os, sys
import pathlib
from threading import Thread, Lock
import concurrent.futures
import time
from collections.abc import Iterable
import cv2
from pdf2image import convert_from_path
from PIL import Image, ImageEnhance, ImageFilter, ImageOps
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
path = '/home/ngoni97/Documents/Documents/MATHEMATICS/Principia Mathematica/Principia_Mathematica [volume.I] alfred_north_whitehead x betrand_russell.pdf'
folder = '/home/ngoni97/Documents/Documents/File Manager with ML/train_test_text_files/images_test/Original Images'
#pages = convert_from_path(path, last_page=10, dpi=300, output_folder=folder, fmt='png')
pages = convert_from_path(path, last_page=10, dpi=300, fmt='png')

In [4]:
pages

[<PIL.PngImagePlugin.PngImageFile image mode=RGB size=2542x2894>,
 <PIL.PngImagePlugin.PngImageFile image mode=RGB size=1962x2861>,
 <PIL.PngImagePlugin.PngImageFile image mode=RGB size=1874x2802>,
 <PIL.PngImagePlugin.PngImageFile image mode=RGB size=1884x2813>,
 <PIL.PngImagePlugin.PngImageFile image mode=RGB size=1845x2781>,
 <PIL.PngImagePlugin.PngImageFile image mode=RGB size=1845x2781>,
 <PIL.PngImagePlugin.PngImageFile image mode=RGB size=1844x2784>,
 <PIL.PngImagePlugin.PngImageFile image mode=RGB size=1844x2780>,
 <PIL.PngImagePlugin.PngImageFile image mode=RGB size=1842x2779>,
 <PIL.PngImagePlugin.PngImageFile image mode=RGB size=1843x2775>]

In [3]:
def advanced_image_preprocessing(image):
    """
    Advanced image preprocessing for better OCR results
    """
    # Convert to grayscale if needed
    if image.mode != 'L':
        image = image.convert('L')
    
    # Convert to numpy array
    img_array = np.array(image)
    
    # 1. Noise reduction
    img_array = cv2.medianBlur(img_array, 3)
    
    # 2. Gaussian blur to smooth
    img_array = cv2.GaussianBlur(img_array, (1, 1), 0)
    
    # 3. Adaptive thresholding for better text separation
    img_array = cv2.adaptiveThreshold(
        img_array, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, 
        cv2.THRESH_BINARY, 11, 2
    )
    
    # 4. Morphological operations to clean up
    kernel = np.ones((1, 1), np.uint8)
    img_array = cv2.morphologyEx(img_array, cv2.MORPH_CLOSE, kernel)
    img_array = cv2.morphologyEx(img_array, cv2.MORPH_OPEN, kernel)
    
    # Convert back to PIL Image
    processed_image = Image.fromarray(img_array)
    
    # 5. PIL-specific enhancements
    # Enhance contrast
    enhancer = ImageEnhance.Contrast(processed_image)
    processed_image = enhancer.enhance(1.2)
    
    # Enhance sharpness
    enhancer = ImageEnhance.Sharpness(processed_image)
    processed_image = enhancer.enhance(1.5)
    
    return processed_image

In [6]:
enhanced_images = []
for img in pages:
    enhanced_images.append(advanced_image_preprocessing(img))

In [7]:
enhanced_images

[<PIL.Image.Image image mode=L size=2542x2894>,
 <PIL.Image.Image image mode=L size=1962x2861>,
 <PIL.Image.Image image mode=L size=1874x2802>,
 <PIL.Image.Image image mode=L size=1884x2813>,
 <PIL.Image.Image image mode=L size=1845x2781>,
 <PIL.Image.Image image mode=L size=1845x2781>,
 <PIL.Image.Image image mode=L size=1844x2784>,
 <PIL.Image.Image image mode=L size=1844x2780>,
 <PIL.Image.Image image mode=L size=1842x2779>,
 <PIL.Image.Image image mode=L size=1843x2775>]

In [9]:
enhanced_images[0].show()

In [18]:
class PdfDataCollector:
    def __init__(self, file_path, pages=None, save_as_text_file=False):
        self.file_path = file_path
        self.pages = pages
        self.save_as_text_file = save_as_text_file
        
        self.file_name = os.path.basename(self.file_path)
        self.book_dict = {}
        self.lock = Lock()  # Thread safety for book_dict
        
        # Validate file exists and is accessible
        if not os.path.exists(file_path):
            raise FileNotFoundError(f"File not found: {file_path}")
        if not os.access(file_path, os.R_OK):
            raise PermissionError(f"Cannot read file: {file_path}")
        
    def remove_characters_before_tokenization(self, sentence, keep_apostrophes=False):
        """Clean text by removing unwanted characters"""
        sentence = (sentence.replace('_', ' ')).replace('-', ' ')
        
        PATTERN = re.compile('[?|$|&|*|%|@|(|)|~]')
        pattern = re.compile('[^a-zA-Z]') if not keep_apostrophes else re.compile('[^a-zA-Z\']')
        replacement = r' '
        
        filtered_sentence = re.sub(pattern, replacement, re.sub(PATTERN, replacement, sentence))
        return filtered_sentence.lower()
    
    def advanced_image_preprocessing(self, image):
        """
        Advanced image preprocessing for better OCR results
        """
        # Convert to grayscale if needed
        if image.mode != 'L':
            image = image.convert('L')
        
        # Convert to numpy array
        img_array = np.array(image)
        
        # 1. Noise reduction
        img_array = cv2.medianBlur(img_array, 3)
        
        # 2. Gaussian blur to smooth
        img_array = cv2.GaussianBlur(img_array, (1, 1), 0)
        
        # 3. Adaptive thresholding for better text separation
        img_array = cv2.adaptiveThreshold(
            img_array, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, 
            cv2.THRESH_BINARY, 11, 2
        )
        
        # 4. Morphological operations to clean up
        kernel = np.ones((1, 1), np.uint8)
        img_array = cv2.morphologyEx(img_array, cv2.MORPH_CLOSE, kernel)
        img_array = cv2.morphologyEx(img_array, cv2.MORPH_OPEN, kernel)
        
        # Convert back to PIL Image
        processed_image = Image.fromarray(img_array)
        
        # 5. PIL-specific enhancements
        # Enhance contrast
        enhancer = ImageEnhance.Contrast(processed_image)
        processed_image = enhancer.enhance(1.2)
        
        # Enhance sharpness
        enhancer = ImageEnhance.Sharpness(processed_image)
        processed_image = enhancer.enhance(1.5)
        
        return processed_image
    
    def convert_pdf_to_images(self, file_path, *, prefix='page', fmt='PNG'):
        # original images
        images = convert_from_path(file_path, last_page=self.pages, dpi=300, fmt='png')
        Images = {} # enhanced images
        print(images)

        destn = '/home/ngoni97/Documents/Documents/File Manager with ML/train_test_text_files/images_test'
        if not os.path.exists(destn):
            os.makedirs(destn, exist_ok=True)
        original_images = os.path.join(destn, 'Original Images')
        if not os.path.exists(original_images):
            os.makedirs(original_images, exist_ok=True)
        self.enhanced_images = os.path.join(destn, 'Enhanced Images')
        if not os.path.exists(self.enhanced_images):
            os.makedirs(self.enhanced_images)

        # creating folders corresponding to each book
        self.saved_images_book_path = os.path.join(self.enhanced_images, os.path.basename(file_path))
        if not os.path.exists(self.saved_images_book_path):
            os.makedirs(self.saved_images_book_path)

        for i, img in enumerate(images):
            Images[f"{prefix}_{i:04d}.{fmt.lower()}"] = self.advanced_image_processing(img)
            Img = self.advanced_image_preprocessing(img)
            file_name = f"{prefix}_{i:04d}.{fmt.lower()}"
            file_path = os.path.join(self.saved_images_book_path, file_name)
            Img.save(file_path, fmt)
            self.read_ocr(i, file_path) # read the single page

                # Combine text from all pages
        text = ""
        for key in sorted(self.book_dict.keys()):
            text += self.book_dict[key]
        
        self.Text = text

        pass 
           
    def read_ocr(self, page_num, book_path):
        """Read OCR text from a specific page - thread-safe version"""
        __text = ""
        doc = None
        try:
            # Open document with error checking
            doc = pymupdf.open(book_path) # for one page documents
            if doc is None:
                return ""
            
            try:
                text_page = doc.get_textpage_ocr(dpi=150, full=True)  # Reduced DPI for stability
                if text_page:
                    __text = text_page.extractTEXT()
                    # Clean up textpage
                    del text_page
            except Exception:
                pass
                
        except Exception:
            pass
        finally:
            # Always close the document
            if doc:
                try:
                    doc.close()
                except:
                    pass
        
        # Thread-safe dictionary update
        with self.lock:
            self.book_dict[page_num] = __text
        
        return __text
    
    def get_document(self, filename):
        """Extract text from PDF - handles both digital and OCR PDFs"""
        self.Text = ""
        
        # Validate file before processing
        if not os.path.exists(filename):
            return ""
        
        # Setup directory structure
        try:
            self.main_folder_path = os.path.basename(os.path.dirname(os.path.dirname(filename)))
            self.parent_directory = os.path.join(
                '/home/ngoni97/Documents/Documents/File Manager with ML/train_test_text_files', 
                self.main_folder_path
            )
            
            if not os.path.exists(self.parent_directory):
                os.makedirs(self.parent_directory, exist_ok=True)
        except Exception:
            pass
        
        # Create text file path
        base_name = os.path.splitext(os.path.basename(filename))[0]
        text_file_path = os.path.join(self.parent_directory, f"{base_name}.txt")
        
        digital_pages = 0
        ocr_pages = 0
        doc = None
        
        try:
            # Open document with explicit error handling
            doc = pymupdf.open(filename)
            if doc is None:
                return ""
            
            doc_length = len(doc)
            if doc_length == 0:
                return ""
            
            pages_to_process = min(self.pages or doc_length, doc_length)
            
            # Test first few pages to determine document type (more conservative)
            test_pages = min(3, doc_length)  # Reduced from 5 to 3
            
            for page_num in range(test_pages):
                try:
                    page = doc[page_num]
                    if page is None:
                        continue
                    
                    # Test for digital text first (safer)
                    try:
                        text = page.get_text()
                        if text and text.strip():
                            digital_pages += 1
                            break
                    except Exception:
                        pass
                    
                    # Only test OCR if no digital text found
                    if digital_pages == 0:
                        try:
                            # Use lighter OCR test
                            if hasattr(page, 'get_textpage_ocr'):
                                ocr_pages += 1
                                break
                        except Exception:
                            pass
                            
                except Exception:
                    continue
            
            # Process based on document type
            if digital_pages > 0:
                # Digital PDF processing (safer)
                for page_num in range(pages_to_process):
                    try:
                        page = doc[page_num]
                        if page is None:
                            continue
                        text = page.get_text()
                        if text:
                            self.Text += text
                    except Exception:
                        continue
            
            elif ocr_pages > 0:
                # OCR processing with reduced concurrency
                doc.close()  # Close before OCR to prevent conflicts
                doc = None
                self.multithreading_ocr(min(pages_to_process, 5), filename)  # Limit OCR pages
            
        except Exception:
            pass
        finally:
            # Always close the document
            if doc:
                try:
                    doc.close()
                except:
                    pass
        
        # Save as text file if requested
        if self.save_as_text_file and self.Text:
            try:
                with open(text_file_path, 'w', encoding='utf-8') as text_file:
                    text_file.write(self.Text)
            except Exception:
                pass
        
        return self.Text
    
    def returnText(self):
        """Return both raw and cleaned text"""
        self._text = self.get_document(self.file_path)
        self.__text = self.remove_characters_before_tokenization(self._text)
        return self._text, self.__text
    
    def get_bookmarks(self, filepath):
        """Extract bookmarks from PDF"""
        self.bookmarks = {}
        try:
            with pymupdf.open(filepath) as file:
                toc = file.get_toc()
                for level, title, page in toc:
                    self.bookmarks[page] = title
        except Exception:
            pass
        return self.bookmarks


In [19]:
path = '/home/ngoni97/Documents/MATHEMATICS/Principia Mathematica/Principia_Mathematica [volume.I] alfred_north_whitehead x betrand_russell.pdf'
test = PdfDataCollector(path, 10)
Text, text = test.returnText()
print(text)